In [1]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('research_traffic.csv')
df.shape

(140000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count_of_Source_IP        int64
Port_Count                int64
Pair_Count_Ratio        float64
Packet_Count_Diff         int64
Lookup_Count_Diff         int64
Protocol                  int64
Average_Packet_Count    float64
Average_Byte_Count      float64
Packet_Std_Dev          float64
Byte_Std_Dev            float64
Duration_per_Flow       float64
Label                     int64
dtype: object
70000
70000


,Count_of_Source_IP,Port_Count,Pair_Count_Ratio,Packet_Count_Diff,Lookup_Count_Diff,Protocol,Average_Packet_Count,Average_Byte_Count,Packet_Std_Dev,Byte_Std_Dev,Duration_per_Flow,Label
0,2,2,1.0,436913,1,6,145637.6667,3.567915e+09,24719.5,5.339086e+09,1.258000,0
1,2,2,1.0,436912,1,6,145637.3333,3.567915e+09,24721.0,5.339086e+09,1.254000,0
2,2,2,1.0,1006277,1,6,335425.6667,8.291520e+09,61289.5,1.240812e+10,2.587333,0
3,2,2,0.0,1,1006280,6,335426.0000,8.291520e+09,61288.0,1.240812e+10,2.592000,0
4,2,2,1.0,1605562,6,6,535187.3333,1.321503e+10,97006.0,1.977597e+10,3.926000,0


In [3]:
# obj_cols=df_XGB.dtypes[df_XGB.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df_XGB[obj_cols] = df_XGB[obj_cols].astype(str)
# df_XGB[obj_cols] = df_XGB[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df_XGB.dtypes)
# df_XGB.head()

In [4]:
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score, make_scorer
from sklearn.metrics import f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, cross_validate

y = df.Label
X = df.drop(['Label'],axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

# X = np.array(X)
# y = np.array(y)

xgb_model = xgb.XGBClassifier(objective='binary:logistic', seed=42)

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

Dataset normalized using MinMaxScaler.


In [5]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []
start = datetime.now()

# Perform cross-validation manually
for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv.iloc[train_index], y_train_cv.iloc[test_index]

    # Fit the model
    xgb_model.fit(X_train, y_train)

    # Predict on training and test data
    y_train_pred = xgb_model.predict(X_train)
    y_test_pred = xgb_model.predict(X_test)

    # Compute confusion matrix for the test data
    cm = confusion_matrix(y_test, y_test_pred)
    confusion_matrices.append(cm)

    # Compute and store scores
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    precisions.append(precision_score(y_test, y_test_pred))
    recalls.append(recall_score(y_test, y_test_pred))
    f1_scores.append(f1_score(y_test, y_test_pred))

# Test the final model on unseen data
y_unseen_pred = xgb_model.predict(X_unseen_test)
unseen_accuracy = accuracy_score(y_unseen_test, y_unseen_pred)
unseen_precision = precision_score(y_unseen_test, y_unseen_pred)
unseen_recall = recall_score(y_unseen_test, y_unseen_pred)
unseen_f1 = f1_score(y_unseen_test, y_unseen_pred)
unseen_conf_matrix = confusion_matrix(y_unseen_test, y_unseen_pred)
end = datetime.now()

In [6]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")
print("excution time: ", end-start)

Fold 1:
Training Accuracy: 0.9886507936507937
Testing Accuracy: 0.988125
Precision: 0.9799789251844047
Recall: 0.996606536881586
F1-Score: 0.9882227928805455
Confusion Matrix:
[[5487  114]
 [  19 5580]]

Fold 2:
Training Accuracy: 0.9891071428571429
Testing Accuracy: 0.9861607142857143
Precision: 0.9774528800422758
Recall: 0.995157819225251
F1-Score: 0.9862258953168044
Confusion Matrix:
[[5496  128]
 [  27 5549]]

Fold 3:
Training Accuracy: 0.9889682539682539
Testing Accuracy: 0.9885714285714285
Precision: 0.9788304702411939
Recall: 0.9989374889321764
F1-Score: 0.9887817703768624
Confusion Matrix:
[[5431  122]
 [   6 5641]]

Fold 4:
Training Accuracy: 0.9889682539682539
Testing Accuracy: 0.9877678571428572
Precision: 0.9793704428241152
Recall: 0.9962011577424024
F1-Score: 0.9877141063581741
Confusion Matrix:
[[5556  116]
 [  21 5507]]

Fold 5:
Training Accuracy: 0.9886904761904762
Testing Accuracy: 0.9883035714285714
Precision: 0.9804955192409067
Recall: 0.9964285714285714
F1-Score: 0.

In [7]:
# import pickle
# with open('xgb_binary.pkl', 'wb') as file:
#     pickle.dump(xgb_model, file)